In [1]:
from google.colab import drive
import os
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install tensorflow
!pip install tensorflow_datasets
!pip install transformers
!pip install sentencepiece
!pip install tf_sentencepiece
!pip install rouge-score

     |████████████████████████████████| 1.4MB 13.3MB/s 
     |████████████████████████████████| 890kB 50.4MB/s 
     |████████████████████████████████| 2.9MB 52.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=8d9f04584023bc1e70c1f8a2bd8acb59544ae3fbe26415a9bbd8b10f84a2ebc3
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 1.1MB 14.0MB/s 
     |████████████████████████████████| 2.1MB 12.9MB/s 


In [4]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
import torch
from torch.utils.data import Dataset, DataLoader
import sentencepiece as spm
from transformers import TFT5ForConditionalGeneration, TrainingArguments, Trainer
from transformers import T5ForConditionalGeneration, T5Tokenizer, AdamW
import dill
from pathlib import Path
import tqdm
from tqdm import trange
from time import sleep

In [5]:
 (ds_train, ds_validation), ds_info = tfds.load(
    'cnn_dailymail',
    split = ['train', 'validation'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

INFO:absl:No config specified, defaulting to first: cnn_dailymail/plain_text
INFO:absl:Load pre-computed DatasetInfo (eg: splits, num examples,...) from GCS: cnn_dailymail/plain_text/3.0.0
INFO:absl:Load dataset info from /tmp/tmpb7pgq5s4tfds
INFO:absl:Field info.description from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.config_name from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.config_description from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.citation from disk and from code do not match. Keeping the one from code.
INFO:absl:Generating dataset cnn_dailymail (/root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0)


INFO:absl:Downloading https://drive.google.com/uc?export=download&id=0BwmD_VLjROrfTHk4NFg2SndKcjQ into /root/tensorflow_datasets/downloads/ucexport_download_id_0BwmD_VLjROrfTHk4NFg2SndKG8BdJPpt2iRo6Dpzz23CByJuAePEilB-pxbcBCHaWDs.tmp.9b2f84d68b3b40ce85659e990c50bfd5...
INFO:absl:Downloading https://drive.google.com/uc?export=download&id=0BwmD_VLjROrfM1BxdkxVaTY2bWs into /root/tensorflow_datasets/downloads/ucexport_download_id_0BwmD_VLjROrfM1BxdkxVaTY2zVV-G71RIXPssrrvSAjt19Cy91r-9CQ2F9DMKA0uFk0.tmp.db72eb917cc24bdc851f4745dbbfd1ad...
INFO:absl:Downloading https://raw.githubusercontent.com/abisee/cnn-dailymail/master/url_lists/all_test.txt into /root/tensorflow_datasets/downloads/raw.gith.com_abis_cnn-dail_mast_url_list_a705_isK790OHOPsIZX-ACsObq_vchU9r5Uduh6ULX3c.txt.tmp.499ed3ae657c4568a2da785978cfbfb7...
INFO:absl:Downloading https://raw.githubusercontent.com/abisee/cnn-dailymail/master/url_lists/all_train.txt into /root/tensorflow_datasets/downloads/raw.gith.com_abis_cnn-dail_mast_url

INFO:absl:Generating split train


Shuffling and writing examples to /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0.incompleteH6YEE3/cnn_dailymail-train.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0.incompleteH6YEE3/cnn_dailymail-train.tfrecord. Shard lengths: [17945, 17944, 17945, 17944, 17945, 17944, 17945, 17944, 17945, 17945, 17944, 17945, 17944, 17945, 17944, 17945]
INFO:absl:Generating split validation


Shuffling and writing examples to /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0.incompleteH6YEE3/cnn_dailymail-validation.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0.incompleteH6YEE3/cnn_dailymail-validation.tfrecord. Shard lengths: [13368]
INFO:absl:Generating split test


Shuffling and writing examples to /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0.incompleteH6YEE3/cnn_dailymail-test.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0.incompleteH6YEE3/cnn_dailymail-test.tfrecord. Shard lengths: [11490]
INFO:absl:Skipping computing stats for mode ComputeStatsMode.SKIP.
INFO:absl:Constructing tf.data.Dataset for split ['train', 'validation'], from /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0


Dataset cnn_dailymail downloaded and prepared to /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0. Subsequent calls will reuse this data.


In [6]:
tf_model = True
model_name = 't5-small'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(torch_device)

Some weights of the model checkpoint at t5-small were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
def strings_from_tensors(tensors):
  texts = [text.numpy().decode() for text, label in tensors]
  labels = [label.numpy().decode() for text, label in tensors]
  return texts, labels

In [8]:
train_texts, train_labels = strings_from_tensors(ds_train)
val_texts, val_labels = strings_from_tensors(ds_validation)

In [9]:
val_texts[1]

"A middle-school teacher in China has inked hundreds of sketches that are beyond be-leaf. Politics teacher Wang Lian, 35,  has created 1000 stunning ink drawings covering subjects as varied as cartoon characters and landscapes to animals, birds according to\xa0the\xa0People's Daily Online. The intricate scribbles on leaves feature Wang's favourite sites across the city of Nanjing, which include the Presidential Palace, Yangtze River Bridge, the ancient Jiming Temple and the Qinhuai River. Natural canvas: Artist and teacher Wang Lian has done hundreds of drawings, like this temple, on leaves she collects in the park and on the streets . Delicate: She uses an ink pen to gently draw the local scenes and buildings on the dried out leaves . 'Although teaching politics is my job, drawing is my passion and hobby,' said Wang. 'I first tried drawing on leaves about 10 years ago and fell in love with it as an art form immediately. 'It's like drawing on very old parchment paper, you have to be re

In [10]:
# val_small = val_texts[:10]

# batch = tokenizer.prepare_seq2seq_batch(src_texts=val_small, truncation=True, padding='longest', return_tensors='pt').to(torch_device)
# summarized = model.generate(**batch)
# summ_text = tokenizer.batch_decode(summarized, skip_special_tokens=True)

In [11]:
# tokenize
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
print('Encodings Generated')
train_labels = tokenizer(train_labels, truncation=True, padding=True)
val_labels = tokenizer(val_labels, truncation=True, padding=True)
print('Labels Generated')

Encodings Generated
Labels Generated


In [12]:
# Dataset Class to be used for Tensorflow
# tf_train_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(train_encodings),
#     train_labels
# ))
# tf_val_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(val_encodings),
#     val_labels
# ))

In [13]:
# Dataset class to be used for Pytorch
class SummaryData(torch.utils.data.Dataset):
    def __init__(self, encodings, labels, size):
        self.encodings = encodings
        self.labels = labels
        self.size = size

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels['input_ids'][idx])
        return item

    def __len__(self):
        return self.size

In [14]:
# Generate predictions using test for CNN/DailyMail Validation set
train_data = SummaryData(train_encodings, train_labels, len(train_texts))
val_data = SummaryData(val_encodings, val_labels, len(val_texts))
train_loader = DataLoader(train_data, batch_size=8, shuffle=True)

In [15]:
# train_path = '/content/drive/MyDrive/NLP Group Project/Data/CNN-DailyMail/train'
# val_path = '/content/drive/MyDrive/NLP Group Project/Data/CNN-DailyMail/validation'

# def save_dataset(dataset, path):
#     if not isinstance(path, Path):
#         path = Path(path)
#     path.mkdir(parents=True, exist_ok=True)
#     torch.save(dataset.encodings, path/"encodings.pkl", pickle_module=dill)
#     torch.save(dataset.labels, path/"labels.pkl", pickle_module=dill)

In [16]:
# save_dataset(train_data, train_path)
# save_dataset(val_data, val_path)

In [17]:
# def load_dataset(path):
#     if not isinstance(path, Path):
#         path = Path(path)
#     examples = torch.load(path/"encodings.pkl", pickle_module=dill)
#     fields = torch.load(path/"labels.pkl", pickle_module=dill)
#     return Dataset(examples, fields)

In [18]:
# Freeze encoder layers for fine-tuning
# for name, param in model.base_model.named_parameters():
#   if name.split('.')[0]=='encoder':
#     param.requires_grad = False
#   else:
#     param.requires_grad = True



In [ ]:
# Train using built in Hugging Face Train Objects

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/NLP Group Project/Models/checkpoints/t5_cnn_model',
    overwrite_output_dir = True,           
    num_train_epochs=10,                       # total number of training epochs
    per_device_train_batch_size=8,                # batch size per device during training
    per_device_eval_batch_size=8,
    # evaluation_strategy = 'epoch'                 # batch size for evaluation
    warmup_steps=2000,                               
    learning_rate=1e-5,                            # strength of weight decay
    logging_dir='./logs',                         # directory for storing logs
    logging_steps=1000,
    save_steps = 1000,
    save_total_limit = 3

)

trainer = Trainer(
    model=model,                                  # the instantiated model to be trained
    args=training_args,                           # training arguments, defined above
    train_dataset=train_data,                     # training dataset
    eval_dataset=val_data,
)

trainer.train()
model.save_pretrained('/content/drive/MyDrive/NLP Group Project/Models/t5_cnn_model_1')

Step,Training Loss
1000,9.304989
2000,0.659176
3000,0.603898
4000,0.597796
5000,0.582566
6000,0.582235
7000,0.583337
8000,0.580129
9000,0.579815
10000,0.580181


In [ ]:
# Train Model in Pytorch
# model.train()

# optim = AdamW(model.parameters(), lr=5e-5)
# train_loader = DataLoader(val_data, batch_size=8, shuffle=True)

# for epoch in range(3):
#   with tqdm.tqdm(train_loader, unit="batch") as train_epoch:
#     for batch in train_epoch:
#       optim.zero_grad()
#       input_ids = batch['input_ids'].to(torch_device)
#       attention_mask = batch['attention_mask'].to(torch_device)
#       labels = batch['labels'].to(torch_device)
#       outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#       loss = outputs[0]
#       print(loss)
#       loss.backward()
#       optim.step()
#       sleep(0.1)

# model.eval()

In [ ]:
# Train Tensorflow Model
# optimizer = keras.optimizers.Adam(learning_rate=5e-5)
# model.compile(optimizer=optimizer, loss=model.compute_loss)
# model.fit(tf_train_dataset.shuffle(10000).batch(8), epochs=5,
#           batch_size=8, validation_data=val_dataset)

In [ ]:
model.train()
val_predictions = []
val_sample = val_texts[:99]
for text in val_sample:
  batch = tokenizer.prepare_seq2seq_batch(src_texts=[text], truncation=True, padding='longest', return_tensors='pt').to(torch_device)
  summarized = model.generate(**batch)
  summ_text = tokenizer.batch_decode(summarized, skip_special_tokens=True)
  print(summ_text[0])

In [ ]:
for i, sum in enumerate(val_predictions):
  print(i, sum)
